In [42]:
from urllib.parse import urlencode
import requests
import time
from datetime import datetime
import json
import pandas as pd

In [2]:
BASE_URL = "http://history.openweathermap.org/data/2.5/history/city?"
API_KEY = open("../wind_farm/api_key.txt").read()

In [33]:
def get_history_weather(params):
    """
    Call OpenWeatherMap historical weather API by coordinates
    Returns JSON file
    """
    query_dict = {"lat": params["lat"],
                  "lon": params["lon"],
                  "type": "hour",
                  "start": params["start"],
                  "end": params["end"],
                  "appid": API_KEY}

    query_str = urlencode(query_dict)

    #print(BASE_URL+query_str)
    api_request = requests.get(BASE_URL+query_str)
    
    return api_request.json()


def get_wind_history(params):
    
    req = get_history_weather(params)
    

In [7]:
params = {"lat": 47.6062,
         "lon": -122.3321,
         "type": "hour",
         "start": int(round(datetime(2020,3,31).timestamp(),0)),
         "end": int(round(datetime(2020,4,1).timestamp(),0))}

In [31]:
req = get_history_weather(params)

http://history.openweathermap.org/data/2.5/history/city?lat=47.6062&lon=-122.3321&type=hour&start=1585638000&end=1585724400&appid=6bce826503c879f393a3f2f7fd8e35b7


In [39]:
req

{'message': 'Count: 25',
 'cod': '200',
 'city_id': 5809844,
 'calctime': 0.00394437,
 'cnt': 25,
 'list': [{'dt': 1585638000,
   'main': {'temp': 277.58,
    'feels_like': 274.49,
    'pressure': 1014,
    'humidity': 86,
    'temp_min': 276.15,
    'temp_max': 278.71},
   'wind': {'speed': 2.1, 'deg': 190},
   'clouds': {'all': 75},
   'weather': [{'id': 500,
     'main': 'Rain',
     'description': 'light rain',
     'icon': '10n'}],
   'rain': {'1h': 0.25}},
  {'dt': 1585641600,
   'main': {'temp': 277.55,
    'feels_like': 274.1,
    'pressure': 1013,
    'humidity': 86,
    'temp_min': 276.15,
    'temp_max': 278.71},
   'wind': {'speed': 2.6, 'deg': 160},
   'clouds': {'all': 90},
   'weather': [{'id': 500,
     'main': 'Rain',
     'description': 'light rain',
     'icon': '10n'}],
   'rain': {'1h': 0.68}},
  {'dt': 1585645200,
   'main': {'temp': 277.44,
    'feels_like': 273.97,
    'pressure': 1014,
    'humidity': 86,
    'temp_min': 275.93,
    'temp_max': 278.71},
   'win

In [72]:
df = pd.DataFrame(data = req["list"])
df.head()

,dt,main,wind,clouds,weather,rain
0,1585638000,"{'temp': 277.58, 'feels_like': 274.49, 'pressu...","{'speed': 2.1, 'deg': 190}",{'all': 75},"[{'id': 500, 'main': 'Rain', 'description': 'l...",{'1h': 0.25}
1,1585641600,"{'temp': 277.55, 'feels_like': 274.1, 'pressur...","{'speed': 2.6, 'deg': 160}",{'all': 90},"[{'id': 500, 'main': 'Rain', 'description': 'l...",{'1h': 0.68}
2,1585645200,"{'temp': 277.44, 'feels_like': 273.97, 'pressu...","{'speed': 2.6, 'deg': 160}",{'all': 75},"[{'id': 500, 'main': 'Rain', 'description': 'l...",{'1h': 0.25}
3,1585648800,"{'temp': 277.35, 'feels_like': 273.17, 'pressu...","{'speed': 3.6, 'deg': 170}",{'all': 75},"[{'id': 803, 'main': 'Clouds', 'description': ...",NaN
4,1585652400,"{'temp': 277.24, 'feels_like': 274.09, 'pressu...","{'speed': 2.1, 'deg': 190}",{'all': 90},"[{'id': 500, 'main': 'Rain', 'description': 'l...",{'1h': 0.3}


In [78]:
df_return = pd.DataFrame(data = df['dt'])

for col in df.columns[1:]:
    if isinstance(df[col][0], dict):
        temp_df = df[col].apply(pd.Series)
    elif isinstance(df[col][0], list):
        temp_df = df[col].apply(pd.Series)[0].apply(pd.Series)
    else:
        temp_df = df[col]
    df_return = df_return.join(temp_df)

In [80]:
def get_wind_history(params):
    """
    Extracts wind speed/direction data from JSON file
    Returns dataframe
    """

    weather_json = get_history_weather(params)

    df = pd.DataFrame(data = weather_json["list"])

    df_return = pd.DataFrame(data = df["dt"])
    
    for col in df.columns[1:]:
        if isinstance(df[col][0], dict):
            temp_df = df[col].apply(pd.Series)
        elif isinstance(df[col][0], list):
            temp_df = df[col].apply(pd.Series)[0].apply(pd.Series)
        else:
            temp_df = df[col]
        df_return = df_return.join(temp_df)
    
    df_return["city_id"] = weather_json["city_id"]
    
    return df_return

In [81]:
get_wind_history(params)

,dt,temp,feels_like,pressure,humidity,temp_min,temp_max,speed,deg,all,id,main,description,icon,0,1h,city_id
0,1585638000,277.58,274.49,1014.0,86.0,276.15,278.71,2.10,190.0,75,500,Rain,light rain,10n,NaN,0.25,5809844
1,1585641600,277.55,274.10,1013.0,86.0,276.15,278.71,2.60,160.0,90,500,Rain,light rain,10n,NaN,0.68,5809844
2,1585645200,277.44,273.97,1014.0,86.0,275.93,278.71,2.60,160.0,75,500,Rain,light rain,10n,NaN,0.25,5809844
3,1585648800,277.35,273.17,1014.0,86.0,275.93,278.71,3.60,170.0,75,803,Clouds,broken clouds,04n,NaN,NaN,5809844
4,1585652400,277.24,274.09,1014.0,86.0,275.93,278.71,2.10,190.0,90,500,Rain,light rain,10n,NaN,0.30,5809844
5,1585656000,276.92,274.32,1014.0,93.0,275.37,278.15,1.50,250.0,90,500,Rain,light rain,10n,NaN,0.30,5809844
6,1585659600,276.70,273.31,1014.0,80.0,275.15,278.15,2.10,0.0,90,804,Clouds,overcast clouds,04n,NaN,NaN,5809844
7,1585663200,276.55,272.44,1015.0,80.0,274.82,278.15,3.10,180.0,90,804,Clouds,overcast clouds,04d,NaN,NaN,5809844
8,1585666800,276.71,272.97,1015.0,80.0,275.15,278.15,2.60,220.0,90,804,Clouds,overcast clouds,04d,NaN,NaN,5809844
9,1585670400,277.41,273.77,1016.0,80.0,275.93,278.71,2.60,200.0,90,804,Clouds,overcast clouds,04d,NaN,NaN,5809844


In [61]:
df.weather[24]

[{'id': 804,
  'main': 'Clouds',
  'description': 'overcast clouds',
  'icon': '04n'}]